In [14]:
import argparse
from pathlib import Path
import sys

wd = Path('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/workflows/BB-NF_17m')
sys.path.append('/Users/alex/Documents/Work/UWyo/Research/Flux Pipeline Project/Eddypro-ec-testing/python')
from eddyproconfigeditor import EddyproConfigEditor

In [15]:
arcc_wd = Path('/project/eddycovworkflow/afox18/Platinum-EddyPro7/workflow/BB-NF_17m')
data_dir = Path('/gscratch/afox18/eddycovworkflow/InputData/Chimney')

In [16]:
template = EddyproConfigEditor(wd / 'ini/BB-NF_17m_template.eddypro')

In [18]:
template.Proj.set_metadata(arcc_wd / 'ini/BB-NF_17m.metadata')
template.Proj.set_biomet(data_dir / 'biomet/BB-NF/EC/17m/EddyPro_Biomet')
template.Proj.set_project_name('BB-NF_17m')

UnboundLocalError: cannot access local variable 'subfolders' where it is not associated with a value

In [ ]:
template.Basic.set_raw_data(path=data_dir / 'raw/BB-NF/EC/17m/Calibrated', fmt='yyyy_mm_dd_HHMM.dat', subfolders=False)
template.Basic.set_out_path(path=arcc_wd / 'output')
template.Basic.set_project_date_range(start='all_available', end='all_available')
template.Basic.set_missing_samples_allowance(pct=10)
template.Basic.set_flux_averaging_interval(minutes=30)
template.Basic.set_north_reference(method='mag')
template.Basic.set_output_id(output_id='template')

In [ ]:
template.Adv.Proc.set_wind_speed_measurement_offsets(0, 0, 0)
template.Adv.Proc.set_axis_rotations_for_tilt_correction(
    method='planar_fit',
    configure_planar_fit_settings_kwargs=dict(
        w_max=0.5,
        u_min=0.5,
        num_per_sector_min=30,
        start='all_available',
        end='all_available',
        fix_method='CW',
        north_offset=0,
        sectors=[(False, 90)]*4
    )
)
template.Adv.Proc.set_turbulent_fluctuations(detrend_method='block')
template.Adv.Proc.set_timelag_compensations(method='covariance_maximization_with_default')
template.Adv.Proc.set_compensation_of_density_fluctuations(
    burba_method='multi',
    day_bot=[3.0935, -0.0819, 0.0018, -0.3601]
    day_top=[0.5773, -0.0107, 0.0012, -0.0914]
    day_spar=[0.7714, -0.0154, 0.0011, -0.1164]
    night_bot=[2.2022, -0.122, 0, -0.3001]
    night_top=[-0.2505, -0.0303, 0, 0.0556]
    night_spar=[0.0219, -0.0361, 0, 0.0145]
)